In [2]:
# table IV in the paper

import os
import json
from prettytable import PrettyTable # type: ignore


modelList = (
    'ESTMD', 'DSTMD', # backbone model
    'STMDPlus',  # with contrast patheway
    'FeedbackSTMD', # with Feedback pathway 1
    'ApgSTMD',  # with attention and prediction mechanism
    'FSTMD',  # with Feedback pathway 2
    'FracSTMD',   # backbone model
    'RPFC', # 
    'ZBS', # Deep Learning-based Unsupervised Algorithm
    'STMDNet',   # STMDNet
    'STMDNetF',  # STMDNet-F
    )

def get_dataset__info(FPS):
    FPS_PARA = 240/FPS

    datasetInfo = {
        f'GX010071-1_{FPS}Hz': list(range(int(1300/FPS_PARA))),
        f'GX010220-1_{FPS}Hz': list(range(int(1300/FPS_PARA))),
        f'GX010228-1_{FPS}Hz': list(range(int(1300/FPS_PARA))),
        f'GX010230-1_{FPS}Hz': list(range(int(2400/FPS_PARA))),
        f'GX010231-1_{FPS}Hz': list(range(int(2400/FPS_PARA))),
        f'GX010241-1_{FPS}Hz': list(range(int(3600/FPS_PARA))),
        f'GX010250-1_{FPS}Hz': list(range(int(2000/FPS_PARA))),
        f'GX010266-1_{FPS}Hz': list(range(int(2400/FPS_PARA))),
        f'GX010290-1_{FPS}Hz': list(range(int(1300/FPS_PARA))),
        f'GX010291-1_{FPS}Hz': list(range(int(1300/FPS_PARA))),
        f'GX010303-1_{FPS}Hz': list(range(int(2400/FPS_PARA))),
        f'GX010307-1_{FPS}Hz': list(range(int(1000/FPS_PARA))),
        f'GX010315-1_{FPS}Hz': list(range(int(1000/FPS_PARA))),
        f'GX010321-1_{FPS}Hz': list(range(int(1000/FPS_PARA))),
        f'GX010322-1_{FPS}Hz': list(range(int(1300/FPS_PARA))),
        f'GX010327-1_{FPS}Hz': list(range(int(900/FPS_PARA))),
        f'GX010335-1_{FPS}Hz': list(range(int(1300/FPS_PARA))),
        f'GX010336-1_{FPS}Hz': list(range(int(1000/FPS_PARA))),
        f'GX010337-1_{FPS}Hz': list(range(int(700/FPS_PARA))),
    }

    return datasetInfo

def get_dataset():

    datasetInfo = {
    'GX010071-1': list(range(1300)),
    'GX010220-1': list(range(1300)),
    'GX010228-1': list(range(1300)),
    'GX010230-1': list(range(2400)),
    'GX010231-1': list(range(2400)),
    'GX010241-1': list(range(3600)),
    'GX010250-1': list(range(2000)),
    'GX010266-1': list(range(2400)),
    'GX010290-1': list(range(1300)),
    'GX010291-1': list(range(1300)),
    'GX010303-1': list(range(2400)),
    'GX010307-1': list(range(1000)),
    'GX010315-1': list(range(1000)),
    'GX010321-1': list(range(1000)),
    'GX010322-1': list(range(1300)),
    'GX010327-1': list(range(900)),
    'GX010335-1': list(range(1300)),
    'GX010336-1': list(range(1000)),
    'GX010337-1': list(range(700)),
}

    return datasetInfo

def get_result_table(FPS):
    if FPS == 240:
        datasetInfo = get_dataset()
    else:
        datasetInfo = get_dataset__info(FPS)

    averageResult = {}
    datasetLen = len(datasetInfo)
    
    evaluateResultFolder = os.path.join(
        '../result', f'RIST_{FPS}Hz',
        )

    for modelName in modelList:

        modelResult = {
            'mAUC'  : 0,
            'AP'    : 0,
            'F1'    : 0,
            'AR'    : 0,
            'mTime'   : 0,}
        
        for datasetName in datasetInfo.keys():
        
            try:
                with open(os.path.join(evaluateResultFolder, datasetName, modelName+'evaluate.json'), 'r') as f:
                    _data = json.load(f)
                modelResult['mAUC'] += _data['AUC'] / datasetLen * 100
                modelResult['AP'] += _data['AP'] / datasetLen * 100
                modelResult['F1'] += _data['F1'] / datasetLen * 100
                modelResult['AR'] += _data['AR'] / datasetLen * 100
                modelResult['mTime'] += _data['timePerImage'] / datasetLen * 1000
            except (FileNotFoundError, json.JSONDecodeError) as e:
                pass

        averageResult[modelName] = modelResult

    return averageResult
    
def get_result():
    result60 = get_result_table(60)
    result120 = get_result_table(120)
    result240 = get_result_table(240)

    with open('result_in_multi_Hz.json', 'w') as json_file:
        json.dump({'60Hz': result60, '120Hz': result120, '240Hz': result240}, json_file, indent=2)

def show_result_table():
    
    with open('result_in_multi_Hz.json', 'r') as f:
        _data = json.load(f)
        result60 = _data['60Hz']
        result120 = _data['120Hz']
        result240 = _data['240Hz']

    # Prepare the tables for AUC, mR, and time per image
    _table = PrettyTable()

    # Initialize the tables with the model names as rows
    _table.field_names = ["ModelName", 
                    "mAUC-240Hz", "AP-240Hz", "F1-240Hz", 
                    "mAUC-120Hz", "AP-120Hz", "F1-120Hz", 
                    "mAUC-60Hz", "AP-60Hz", "F1-60Hz", 
                    "mTime-240Hz", #"mTime-120Hz", "mTime-60Hz",
                    ]
    
    
    for modelName in modelList:
        modelRes = [
            modelName if modelName != 'STMDNetF' else 'STMDNet-F',
            f"{result240[modelName]['mAUC']:.2f}" if result240[modelName]['mAUC'] >= 0 else '-',  
            f"{result240[modelName]['AP']:.2f}", 
            f"{result240[modelName]['F1']:.2f}",
            f"{result120[modelName]['mAUC']:.2f}" if result120[modelName]['mAUC'] >= 0 else '-',  
            f"{result120[modelName]['AP']:.2f}", 
            f"{result120[modelName]['F1']:.2f}", 
            f"{result60[modelName]['mAUC']:.2f}" if result60[modelName]['mAUC'] >= 0 else '-', 
            f"{result60[modelName]['AP']:.2f}", 
            f"{result60[modelName]['F1']:.2f}", 
            f"{result240[modelName]['mTime']:.2f}", 
            # f"{result120[modelName]['mTime']:.2f}", 
            # f"{result60[modelName]['mTime']:.2f}",
            
            ]
        _table.add_row(modelRes)

    # Print the tables
    print(_table)

if __name__ == '__main__':

    get_result()

    show_result_table()


+--------------+------------+----------+----------+------------+----------+----------+-----------+---------+---------+-------------+
|  ModelName   | mAUC-240Hz | AP-240Hz | F1-240Hz | mAUC-120Hz | AP-120Hz | F1-120Hz | mAUC-60Hz | AP-60Hz | F1-60Hz | mTime-240Hz |
+--------------+------------+----------+----------+------------+----------+----------+-----------+---------+---------+-------------+
|    ESTMD     |   20.47    |   6.52   |   8.25   |    8.03    |   1.60   |   2.13   |    2.01   |   0.47  |   0.58  |    17.83    |
|    DSTMD     |   27.97    |  12.36   |  15.16   |   11.88    |   3.98   |   5.12   |    3.98   |   0.73  |   0.90  |    54.31    |
|   STMDPlus   |   35.42    |  17.86   |  21.54   |   19.68    |   8.71   |  10.84   |    7.23   |   2.04  |   2.73  |    90.58    |
| FeedbackSTMD |   39.27    |  14.82   |  18.39   |   19.19    |   5.70   |   7.46   |    6.71   |   1.35  |   1.83  |    21.21    |
|   ApgSTMD    |   25.86    |   6.03   |   7.13   |   13.46    |   1.